<a href="https://colab.research.google.com/github/hxlinh/Arctic-TILT/blob/main/src/Arctic_TILT_add_separate_chunks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq wandb

In [ ]:
#l login wandb
from google.colab import userdata
import wandb

wandb_token = userdata.get('wandb_api')
wandb.login(key=wandb_token)

try:
    user = wandb.api.viewer()
    print("✅ Đã đăng nhập thành công với Weights & Biases!")
    print("Tên tài khoản:", user.get('username', 'Không xác định'))
except Exception as e:
    print("❌ Chưa đăng nhập hoặc token không hợp lệ.")
    print("Lỗi:", e)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xuanlinh-work (xuanlinh-work-dut) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Đã đăng nhập thành công với Weights & Biases!
Tên tài khoản: xuanlinh-work


In [ ]:
!pip install -qqq datasets huggingface_hub fsspec sentencepiece pytorch_lightning seqeval evaluate transformers==4.37.2
!pip install -qqq --upgrade datasets huggingface_hub fsspec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━

In [ ]:
#l login hf
from google.colab import userdata
from huggingface_hub import login
from huggingface_hub import whoami

hf_token = userdata.get('HF_TOKEN')
login(hf_token)

try:
    info = whoami()
    print("✅ Đã đăng nhập thành công với Hugging Face!")
    print("Tên tài khoản:", info["name"])
    print("Vai trò:", info.get("type", "Không xác định"))
except Exception as e:
    print("❌ Chưa đăng nhập hoặc token không hợp lệ.")
    print("Lỗi:", e)


✅ Đã đăng nhập thành công với Hugging Face!
Tên tài khoản: xlinh
Vai trò: user


In [ ]:
import os
from transformers import AutoTokenizer, AutoConfig
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

from torchvision import transforms
from tqdm.auto import tqdm

## Custom imports
from transformers import AutoModel

## test. Chunked processing

In [ ]:
import math
from typing import Optional, Tuple, List
import torch
import torch.nn as nn

class SparseAttentionMask:
    @staticmethod
    def create_block_diagonal_mask(seq_length: int, block_size: int,
                                 num_blocks: int = None, overlap: int = 0,
                                 device: torch.device = None) -> torch.Tensor:
        if device is None:
            device = torch.device('cpu')

        # Create empty mask (all positions ignored initially)
        mask = torch.full((seq_length, seq_length), float('-inf'), device=device)

        if num_blocks is None:
            num_blocks = math.ceil(seq_length / (block_size - overlap)) if overlap > 0 else math.ceil(seq_length / block_size)

        for block_idx in range(num_blocks):
            # Calculate block boundaries
            start_pos = block_idx * (block_size - overlap) if overlap > 0 else block_idx * block_size
            end_pos = min(start_pos + block_size, seq_length)

            if start_pos >= seq_length:
                break

            # Create attention block - tokens in this block can attend to each other
            mask[start_pos:end_pos, start_pos:end_pos] = 0.0

            # Add neighborhood attention (tokens can attend to adjacent blocks)
            if block_idx > 0:  # Left neighbor
                prev_start = max(0, start_pos - overlap if overlap > 0 else start_pos - block_size)
                mask[start_pos:end_pos, prev_start:start_pos] = 0.0
                mask[prev_start:start_pos, start_pos:end_pos] = 0.0

            if block_idx < num_blocks - 1:  # Right neighbor
                next_end = min(seq_length, end_pos + overlap if overlap > 0 else end_pos + block_size)
                mask[start_pos:end_pos, end_pos:next_end] = 0.0
                mask[end_pos:next_end, start_pos:end_pos] = 0.0

        return mask

    @staticmethod
    def create_neighborhood_diagonal_mask(seq_length: int, window_size: int = 256,
                                        device: torch.device = None) -> torch.Tensor:
        if device is None:
            device = torch.device('cpu')

        mask = torch.full((seq_length, seq_length), float('-inf'), device=device)

        for i in range(seq_length):
            # Define window around position i
            start = max(0, i - window_size // 2)
            end = min(seq_length, i + window_size // 2 + 1)
            mask[i, start:end] = 0.0

        return mask


class ChunkedProcessor:

    def __init__(self, core_chunk_length: int = 1024, chunk_overlap: int = 0):
        self.core_chunk_length = core_chunk_length
        self.chunk_overlap = chunk_overlap

    def create_chunks(self,
                     input_ids: torch.Tensor,
                     attention_mask: torch.Tensor,
                     prefix_length: int = 1) -> Tuple[List[torch.Tensor], List[torch.Tensor], List[int]]:
        batch_size, total_length = input_ids.shape

        # If sequence is shorter than chunk length, return as single chunk
        if total_length <= self.core_chunk_length + prefix_length:
            return [input_ids], [attention_mask], [0]

        chunked_input_ids = []
        chunked_attention_masks = []
        chunk_info = []

        # Extract prefix (typically special tokens like [CLS])
        if prefix_length > 0:
            prefix_ids = input_ids[:, :prefix_length]
            prefix_mask = attention_mask[:, :prefix_length]
            input_content = input_ids[:, prefix_length:]
            mask_content = attention_mask[:, prefix_length:]
        else:
            prefix_ids = None
            prefix_mask = None
            input_content = input_ids
            mask_content = attention_mask

        content_length = input_content.shape[1]
        current_pos = 0

        while current_pos < content_length:
            # Calculate chunk boundaries
            chunk_end = min(current_pos + self.core_chunk_length, content_length)

            # Extract content chunk
            content_chunk = input_content[:, current_pos:chunk_end]
            mask_chunk = mask_content[:, current_pos:chunk_end]

            # Prepend prefix to chunk (except for padding)
            if prefix_ids is not None:
                chunk_input_ids = torch.cat([prefix_ids, content_chunk], dim=1)
                chunk_attention_mask = torch.cat([prefix_mask, mask_chunk], dim=1)
            else:
                chunk_input_ids = content_chunk
                chunk_attention_mask = mask_chunk

            chunked_input_ids.append(chunk_input_ids)
            chunked_attention_masks.append(chunk_attention_mask)
            chunk_info.append(current_pos)

            # Move to next chunk position
            if self.chunk_overlap > 0:
                current_pos = chunk_end - self.chunk_overlap
            else:
                current_pos = chunk_end

            # Break if we've covered all content
            if chunk_end >= content_length:
                break

        return chunked_input_ids, chunked_attention_masks, chunk_info

    def recombine_chunk_outputs(self,
                              chunk_outputs: List[torch.Tensor],
                              prefix_length: int = 1) -> torch.Tensor:
        if len(chunk_outputs) == 1:
            return chunk_outputs[0]

        combined_chunks = []

        for i, chunk_output in enumerate(chunk_outputs):
            if i == 0:
                # Keep first chunk as-is (includes prefix)
                combined_chunks.append(chunk_output)
            else:
                # Remove prefix embeddings from subsequent chunks
                if prefix_length > 0:
                    chunk_without_prefix = chunk_output[:, prefix_length:, :]
                    combined_chunks.append(chunk_without_prefix)
                else:
                    combined_chunks.append(chunk_output)

        # Concatenate all chunks along sequence dimension
        combined_output = torch.cat(combined_chunks, dim=1)
        return combined_output

    def process_long_sequence(self,
                            encoder_func,
                            input_ids: torch.Tensor,
                            attention_mask: torch.Tensor,
                            prefix_length: int = 1,
                            **encoder_kwargs) -> torch.Tensor:

        # Create chunks
        chunked_inputs, chunked_masks, chunk_info = self.create_chunks(
            input_ids, attention_mask, prefix_length
        )

        # Process each chunk through encoder
        chunk_outputs = []
        for chunk_input, chunk_mask in zip(chunked_inputs, chunked_masks):
            # Apply encoder to chunk
            chunk_output = encoder_func(
                input_ids=chunk_input,
                attention_mask=chunk_mask,
                **encoder_kwargs
            )

            # Extract hidden states (handle both tensor and object outputs)
            if hasattr(chunk_output, 'last_hidden_state'):
                chunk_hidden = chunk_output.last_hidden_state
            else:
                chunk_hidden = chunk_output

            chunk_outputs.append(chunk_hidden)

        # Recombine chunk outputs
        combined_output = self.recombine_chunk_outputs(chunk_outputs, prefix_length)

        return combined_output

## 1.1. Preparing the dataset

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

hf_ds = load_dataset("nielsr/funsd-layoutlmv3")
model_name = "t5-base"
## Visual Embedding extractor's parameters
in_channels = 3
num_pool_layers = 3
channels = 16
sampling_ratio = 2
spatial_scale = 48 / 384
output_size = (3,3)
load_weights = True
max_epochs = 2 #l 50 -> 2

## FUNSD Dataset specific
num_classes = 7

## Tokenizer's parameter
model_max_length = 512

t5_config = AutoConfig.from_pretrained(model_name)
## Adding new parameters for Arctic-TILT
t5_config.update(dict(
    in_channels = in_channels,
    num_pool_layers = num_pool_layers,
    channels = channels,
    model_max_length = model_max_length,
    output_size = output_size,
    spatial_scale = spatial_scale,
    sampling_ratio = sampling_ratio,
    use_cache = False,
    load_weights = load_weights,
    lr =  2e-4,
    num_classes = num_classes,
    max_epochs = max_epochs,
    # Arctic-TILT parameters
    use_post_fusion = True,  # Arctic-TILT post-fusion module
    fusion_dropout = 0.1,    # Dropout rate for fusion module
    core_chunk_length = 1024,  # Core chunk length for SLED
    chunk_overlap = 0,         # No overlap between chunks
    use_chunked_processing = True,  # Enable chunked processing
))

README.md:   0%|          | 0.00/770 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/9.54M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/149 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [ ]:
def get_id2label_and_label2id():
    label2id = {'O': 0, 'B-HEADER': 1, 'I-HEADER': 2, 'B-QUESTION': 3, 'I-QUESTION': 4, 'B-ANSWER': 5, 'I-ANSWER': 6}
    id2label = {0: 'O', 1: 'B-HEADER', 2: 'I-HEADER', 3: 'B-QUESTION', 4: 'I-QUESTION', 5: 'B-ANSWER', 6: 'I-ANSWER'}
    return id2label, label2id

def convert_id_to_label(list_of_label):
  return [id2label[x] for x in list_of_label]

## 1.2 ExtFUNSDDs

In [ ]:
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import torch


## This is a dataset referring to the generative problem
## The Dataset class for FUNSD Dataset (and I believe the same would be use for CORD Dataset)
class FUNSDDs(Dataset):

  def __init__(self, ds, tokenizer, max_seq_length:int = 512, pad_token_box = [0, 0, 0, 0], resize_scale = (512, 384), transform = None):

    """
    Args:
      ds (list): list of dict, each dict contains the following keys:
        - image (np.ndarray): the image
        - tokens (list): list of tokens
        - bboxes (list): list of bboxes
        - ner_tags (list): list of ner_tags
      tokenizer (Tokenizer): the tokenizer
      max_seq_length (int, optional): the maximum length of the sequence. Defaults to 512.
      pad_token_box (list, optional): the padding token box. Defaults to [0, 0, 0, 0].
      resize_scale (tuple, optional): the resize scale. Defaults to (512, 384).
      transform (callable, optional): the transform. Defaults to None.
    """

    self.ds = ds
    self.tokenizer = tokenizer
    self.max_seq_length = max_seq_length
    self.pad_token_box = pad_token_box
    self.resize_scale = resize_scale
    self.transform = transform if transform is not None else ToTensor()

  def __len__(self):
    """
    Returns:
      int: the length of the dataset
    """
    return len(self.ds)

  def __getitem__(self, idx):

    """
    Args:
      idx (int): the index of the data to be returned.
    """

    encoding = self.ds[idx]

    resized_image = encoding['image'].copy().resize(self.resize_scale)
    words = encoding['tokens']
    bboxes = encoding['bboxes']
    labels = encoding['ner_tags']

    ## 1. Performing the image pre-processing
    img_tensor = self.transform(resized_image)  ## (3, 384, 512)

    ## 2. Performing the semantic pre-processing
    encoding = self.tokenizer(words, is_split_into_words = True, add_special_tokens = False)

    # pad_token_box = [0, 0, 0, 0]
    max_seq_length = 512

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    ## Note that, there is no need for bboxes, since the model does not use bbox as feature, so no pre-processing of that
    bbox_according_to_tokenizer = [bboxes[i] for i in encoding.word_ids()]
    # labels_according_to_tokenizer = [self.tokenizer(str(labels[i] + 1))['input_ids'][0] for i in encoding.word_ids()]
    #labels_according_to_tokenizer = [self.tokenizer(str(labels[i] + 1))['input_ids'][0] for i, _ in enumerate(labels)]

    # Truncation of token_boxes + token_labels
    special_tokens_count = 1
    if len(input_ids) > max_seq_length - special_tokens_count:
        bbox_according_to_tokenizer = bbox_according_to_tokenizer[: (max_seq_length - special_tokens_count)]
        input_ids = input_ids[: (max_seq_length - special_tokens_count)]
        #labels_according_to_tokenizer = labels_according_to_tokenizer[: (max_seq_length - special_tokens_count)]
        attention_mask = attention_mask[: (max_seq_length - special_tokens_count)]


    ## Padding
    input_ids =  input_ids + [self.tokenizer.eos_token_id]
    bbox_according_to_tokenizer = bbox_according_to_tokenizer + [[1000, 1000, 1000, 1000]]
    #labels_according_to_tokenizer = labels_according_to_tokenizer + [self.tokenizer.eos_token_id] ## For QA, the model requires an end of sentence i.e eos token
    attention_mask = attention_mask + [1]

    pad_length = max_seq_length -  len(input_ids)

    input_ids = input_ids + [self.tokenizer.pad_token_id] * (pad_length)
    bbox_according_to_tokenizer = bbox_according_to_tokenizer + [self.pad_token_box] * (pad_length)
    #labels_according_to_tokenizer = labels_according_to_tokenizer + [self.tokenizer.pad_token_id] * (pad_length)
    attention_mask = attention_mask + [0] * (pad_length)

    ## Converting stuffs to tensor
    input_ids = torch.tensor(input_ids)
    bbox_according_to_tokenizer = torch.tensor(bbox_according_to_tokenizer)
    #labels_according_to_tokenizer = torch.tensor(labels_according_to_tokenizer)
    attention_mask = torch.tensor(attention_mask)

    return {"input_ids" : input_ids,  "labels" : labels, "attention_mask" : attention_mask, "bboxes" : bbox_according_to_tokenizer,  # labels_according_to_tokenizer
            "pixel_values" : img_tensor}


## This is a dataset referring to the extractive problem, I believe the same would be used for CORD Dataset, and this was also a mistake, since I didn't  read the last part of the paper properly
class ExtFUNSDDs(Dataset):
  def __init__(self, ds, tokenizer, max_seq_length:int = 512, pad_token_box = [0, 0, 0, 0], resize_scale = (512, 384), transform = None):

    """
    Args:
      ds (list): list of dict, each dict contains the following keys:
        - image (np.ndarray): the image
        - tokens (list): list of tokens
        - bboxes (list): list of bboxes
        - ner_tags (list): list of ner_tags
      tokenizer (Tokenizer): the tokenizer
      max_seq_length (int, optional): the maximum length of the sequence. Defaults to 512.
      pad_token_box (list, optional): the padding token box. Defaults to [0, 0, 0, 0].
      resize_scale (tuple, optional): the resize scale. Defaults to (512, 384).
      transform (callable, optional): the transform. Defaults to None.
    """

    self.ds = ds
    self.tokenizer = tokenizer
    self.max_seq_length = max_seq_length
    self.pad_token_box = pad_token_box
    self.resize_scale = resize_scale
    self.transform = transform if transform is not None else ToTensor()

  def __len__(self):
    """
    Returns:
      int: the length of the dataset
    """
    return len(self.ds)

  def __getitem__(self, idx):

    """
    Args:
      idx (int): the index of the data to be returned.
    """

    encoding = self.ds[idx]

    resized_image = encoding['image'].copy().resize(self.resize_scale)
    words = encoding['tokens']
    bboxes = encoding['bboxes']
    labels = encoding['ner_tags']

    ## 1. Performing the image pre-processing
    img_tensor = self.transform(resized_image)  ## (3, 384, 512)

    ## 2. Performing the semantic pre-processing
    encoding = self.tokenizer(words, is_split_into_words = True, add_special_tokens = False)

    # pad_token_box = [0, 0, 0, 0]
    max_seq_length = 512

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    ## Note that, there is no need for bboxes, since the model does not use bbox as feature, so no pre-processing of that
    bbox_according_to_tokenizer = [bboxes[i] for i in encoding.word_ids()]
    labels_according_to_tokenizer = [labels[i] for i in encoding.word_ids()]  ## Labels have to be in the numerical format
    #labels_according_to_tokenizer = [self.tokenizer(str(labels[i] + 1))['input_ids'][0] for i, _ in enumerate(labels)]

    # Truncation of token_boxes + token_labels
    special_tokens_count = 1
    if len(input_ids) > max_seq_length - special_tokens_count:
        bbox_according_to_tokenizer = bbox_according_to_tokenizer[: (max_seq_length - special_tokens_count)]
        input_ids = input_ids[: (max_seq_length - special_tokens_count)]
        labels_according_to_tokenizer = labels_according_to_tokenizer[: (max_seq_length - special_tokens_count)]
        attention_mask = attention_mask[: (max_seq_length - special_tokens_count)]


    ## Padding
    input_ids =  input_ids + [self.tokenizer.eos_token_id]
    bbox_according_to_tokenizer = bbox_according_to_tokenizer + [[1000, 1000, 1000, 1000]]
    labels_according_to_tokenizer = labels_according_to_tokenizer + [-100] ## For QA, the model requires an end of sentence i.e eos token
    attention_mask = attention_mask + [1]

    pad_length = max_seq_length -  len(input_ids)

    input_ids = input_ids + [self.tokenizer.pad_token_id] * (pad_length)
    bbox_according_to_tokenizer = bbox_according_to_tokenizer + [self.pad_token_box] * (pad_length)
    labels_according_to_tokenizer = labels_according_to_tokenizer + [-100] * (pad_length)
    attention_mask = attention_mask + [0] * (pad_length)

    ## Converting stuffs to tensor
    input_ids = torch.tensor(input_ids)
    bbox_according_to_tokenizer = torch.tensor(bbox_according_to_tokenizer)
    labels_according_to_tokenizer = torch.tensor(labels_according_to_tokenizer)
    attention_mask = torch.tensor(attention_mask)

    return {"input_ids" : input_ids,  "labels" : labels_according_to_tokenizer, "attention_mask" : attention_mask, "bboxes" : bbox_according_to_tokenizer,  # labels_according_to_tokenizer
            "pixel_values" : img_tensor}

## 1.3. Fusion

In [ ]:
class TiltLayerNorm(nn.Module):
    """
    This is essentially the T5 modification of layer norm, referred to as RMS norm.

    Args:
        dim: the dimension of vectors to be normalized, i.e. the last dimension of
             the input tensor
        eps: small positive value added to computed second moment for numerical
             stability
    """
    def __init__(self, dim: int, eps: float = 1e-6) -> None:
        super().__init__()
        self.w = nn.Parameter(torch.ones(dim))
        self.eps = eps
        self.init_weights()

    def forward(self, inp: torch.Tensor) -> torch.Tensor:
        dtype = inp.dtype
        x = inp.to(torch.float32)
        squared_norm = x.pow(2).mean(dim=-1, keepdim=True)
        x = x * torch.rsqrt(squared_norm + self.eps)
        return self.w * x.to(dtype)

    def init_weights(self, factor: float = 1.0) -> None:
        self.w.data.fill_(factor * 1.0)

class TiltPostFusionModule(nn.Module):
    """
    Introduced in the Arctic-TILT paper.

    Args:
        d_model: dimension of input vectors
        dropout: probability of dropout applied to input embeddings
        layer_norm: the module responsible for input embeddings
    """

    def __init__(self, d_model: int, dropout: float, layer_norm: TiltLayerNorm):
        super().__init__()
        self.layer_norm = layer_norm
        self.to_v = nn.Linear(d_model, d_model, bias=False)
        self.to_out = nn.Linear(d_model, d_model, bias=False)
        self.to_r = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text_queries: torch.Tensor, image_queries: torch.Tensor) -> torch.Tensor:
        """
        Compute module’s forward pass.

        Args:
            text_queries (Tensor): Tensor representing the primary input in the
                                  fusion, which is text-based, or mixed.
            image_queries (Tensor): Tensor representing the secondary input in the
                                   fusion, which is image-based.
        """
        bs, l, d = text_queries.shape
        inputs = torch.stack([text_queries, image_queries], dim=-2)
        inputs = inputs.view(bs * l, 2, d)
        normed_inputs = self.dropout(self.layer_norm(inputs))
        normed_primary_input = normed_inputs[:, 0]
        out: Tensor = self.to_v(normed_inputs.sum(-2))
        out = out + out * self.to_r(normed_primary_input)
        out = self.to_out(out)
        out = out.view(bs, l, d)
        return text_queries + out

## 1.4 Visual backbone

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision.ops import roi_pool

# Convolution block for UNet Encoder
class ConvBlock(nn.Module):
    """
    A Convolutional Block that consists of two convolution layers each followed by
    instance normalization, LeakyReLU activation and dropout.
    """

    def __init__(self, in_chans: int, out_chans: int, drop_prob: float):
        """
        Args:
            in_chans: Number of channels in the input.
            out_chans: Number of channels in the output.
            drop_prob: Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.drop_prob = drop_prob

        self.layers = nn.Sequential(
            nn.Conv2d(in_chans, out_chans, kernel_size=3,
                      padding=1, bias=False),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Dropout2d(drop_prob),
            nn.Conv2d(out_chans, out_chans, kernel_size=3,
                      padding=1, bias=False),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Dropout2d(drop_prob),
            nn.Conv2d(out_chans, out_chans, kernel_size=3,
                      padding=1, bias=False),
            nn.InstanceNorm2d(out_chans),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Dropout2d(drop_prob),
        )

    def forward(self, image: torch.Tensor) -> torch.Tensor:
        """
        Args:
            image: Input 4D tensor of shape `(N, in_chans, H, W)`.
        Returns:
            Output tensor of shape `(N, out_chans, H, W)`.
        """
        return self.layers(image)


# UNet Encoder
class Unet_encoder(nn.Module):

    def __init__(self,
                 in_channels: int = 3,
                 channels: int = 32,
                 num_pool_layers: int = 4,
                 drop_prob: float = 0.0
                 ):
        """
            Args:
                in_chans: Number of channels in the input to the U-Net model.
                out_chans: Number of channels in the output to the U-Net model.
                chans: Number of output channels of the first convolution layer.
                num_pool_layers: Number of down-sampling and up-sampling layers.
                drop_prob: Dropout probability.
        """
        super().__init__()

        self.in_channels = in_channels
        self.channels = channels

        self.num_pool_layers = num_pool_layers
        self.drop_prob = drop_prob

        self.down_sample_layers = nn.ModuleList([
            ConvBlock(in_channels, channels, drop_prob)
        ])
        ch = channels

        for _ in range(num_pool_layers - 1):
            self.down_sample_layers.append(ConvBlock(ch, ch*2, drop_prob))
            ch *= 2

        self.conv = ConvBlock(ch, ch*2, drop_prob)

    def forward(self, image: torch.Tensor) -> torch.Tensor:
        """
        Args:
          Image: Input 4D tensor of shape (Batch Size, in channels, H, W)
        Returns:
          Output tensor of shape (Batch Size, out_channels, H, W)
        """
        output = image

        # Appplying down sample layers
        for num, layer in enumerate(self.down_sample_layers):
            output = layer(output)
            output = F.max_pool2d(output, kernel_size=2, stride=2, padding=0)

        output = self.conv(output)
        return output


# RoI Align, it was a mistake, I assumed RoIPool for RoIALign, but it was not the case

class RoIAlign(nn.Module):
    def __init__(self, output_size=(3, 3), spatial_scale=0.125, sampling_ratio=2):
        super().__init__()

        """
        Args
        output_size: (h, w) of the output feature map
        spatial_scale: ratio of the input feature map height (or w) to the raw image height (or w).
                        Equals the reciprocal of total stride in convolutional layers
        sampling_ratio: number of inputs samples to take for each output sample
        """

        # self.output_size = output_size
        # self.spatial_scale = spatial_scale
        # self.sampling_ratio = sampling_ratio
        self.roi_align = RoIAlign(
            output_size, spatial_scale=spatial_scale, sampling_ratio=sampling_ratio)

    def forward(self, image_embedding, bboxes):
        """
        Args:
          image_embedding: Input 4D tensor of shape (Batch size, in channels, H, W)
          bboxes: Input 3D Tensor of shape (Batch Size, max sequence length, 4) (4 corresponding to xmin, ymin, xmax, ymax)
        Returns:
          feature_maps_bboxes: tensor of shape (batch, max sequence length, in channels, *output_size)
        """

        feature_maps_bboxes = []
        for single_batch_img, single_batch_bbox in zip(image_embedding, bboxes):
            feature_map_single_batch = self.roi_align(input=single_batch_img.unsqueeze(0),
                                                      rois=torch.cat([torch.zeros(single_batch_bbox.shape[0], 1).to(
                                                          single_batch_bbox.device), single_batch_bbox], axis=-1).float()
                                                      )
            feature_maps_bboxes.append(feature_map_single_batch)

        return torch.stack(feature_maps_bboxes, axis=0)


# RoIPool

class RoIPool(nn.Module):

    def __init__(self, output_size=(3, 3), spatial_scale=0.125):
        super().__init__()
        """Args
        output_size: (h, w) of the output feature map
        spatial_scale: ratio of the input feature map height (or w) to the raw image height (or w).
                        Equals the reciprocal of total stride in convolutional layers
        """

        self.output_size = output_size
        self.spatial_scale = spatial_scale
        self.roi_pool = roi_pool

    def forward(self, image_embedding, bboxes):
        """
        Args:
          image_embedding: Input 4D tensor of shape (Batch size, in channels, H, W)
          bboxes: Input 3D Tensor of shape (Batch Size, max sequence length, 4) (4 corresponding to xmin, ymin, xmax, ymax)
        Returns:
          feature_maps_bboxes: tensor of shape (batch, max sequence length, in channels, *output_size)
        """

        feature_maps_bboxes = []
        for single_batch_img, single_batch_bbox in zip(image_embedding, bboxes):
            feature_map_single_batch = self.roi_pool(input=single_batch_img.unsqueeze(0),
                                                     boxes=torch.cat([torch.zeros(single_batch_bbox.shape[0], 1).to(
                                                         single_batch_bbox.device), single_batch_bbox], axis=-1).float(),
                                                     output_size=self.output_size,
                                                     spatial_scale=self.spatial_scale
                                                     )
            feature_maps_bboxes.append(feature_map_single_batch)

        return torch.stack(feature_maps_bboxes, axis=0)

## 1.5 t5

In [ ]:
import torch
import torch.nn as nn
import math
import copy
from transformers.models import t5
from transformers import AutoModel


class T5LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        Construct a layernorm module in the T5 Style. No bias and no subtraction of mean.
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):

        # T5 uses a layer_norm which only scales and doesn't shift, which is also known as Root Mean
        # Square Layer Normalization https://arxiv.org/abs/1910.07467 thus varience is calculated
        # w/o mean and there is no bias. Additionally we want to make sure that the accumulation for
        # half-precision inputs is done in fp32

        variance = hidden_states.to(torch.float32).pow(
            2).mean(-1, keepdim=True)
        hidden_states = hidden_states * \
            torch.rsqrt(variance + self.variance_epsilon)

        # convert into half-precision if necessary
        if self.weight.dtype in [torch.float16, torch.bfloat16]:
            hidden_states = hidden_states.to(self.weight.dtype)

        return self.weight * hidden_states


class T5DenseActDense(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.wi = nn.Linear(config.d_model, config.d_ff, bias=False)
        self.wo = nn.Linear(config.d_ff, config.d_model, bias=False)
        self.dropout = nn.Dropout(config.dropout_rate)
        self.act = nn.ReLU()

    def forward(self, hidden_states):
        hidden_states = self.wi(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.dropout(hidden_states)
        if hidden_states.dtype != self.wo.weight.dtype and self.wo.weight.dtype != torch.int8:
            hidden_states = hidden_states.to(self.wo.weight.dtype)
        hidden_states = self.wo(hidden_states)
        return hidden_states


class T5DenseGatedActDense(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.wi_0 = nn.Linear(config.d_model, config.d_ff, bias=False)
        self.wi_1 = nn.Linear(config.d_model, config.d_ff, bias=False)
        self.wo = nn.Linear(config.d_ff, config.d_model, bias=False)
        self.dropout = nn.Dropout(config.dropout_rate)
        self.act = nn.ReLU()

    def forward(self, hidden_states):
        hidden_gelu = self.act(self.wi_0(hidden_states))
        hidden_linear = self.wi_1(hidden_states)
        hidden_states = hidden_gelu * hidden_linear
        hidden_states = self.dropout(hidden_states)
        if hidden_states.dtype != self.wo.weight.dtype and self.wo.weight.dtype != torch.int8:
            hidden_states = hidden_states.to(self.wo.weight.dtype)
        hidden_states = self.wo(hidden_states)
        return hidden_states


class T5LayerFF(nn.Module):
    def __init__(self, config):
        super().__init__()
        if config.is_gated_act:
            self.DenseReluDense = T5DenseGatedActDense(config)
        else:
            self.DenseReluDense = T5DenseActDense(config)

        self.layer_norm = T5LayerNorm(
            config.d_model, eps=config.layer_norm_epsilon)
        self.dropout = nn.Dropout(config.dropout_rate)

    def forward(self, hidden_states):
        forwarded_states = self.layer_norm(hidden_states)
        forwarded_states = self.DenseReluDense(forwarded_states)
        hidden_states = hidden_states + self.dropout(forwarded_states)
        return hidden_states


class T5Attention(nn.Module):
    def __init__(self, config, has_relative_attention_bias=False):
        super().__init__()
        self.is_decoder = config.is_decoder
        self.has_relative_attention_bias = has_relative_attention_bias
        self.relative_attention_num_buckets = config.relative_attention_num_buckets
        self.relative_attention_max_distance = config.relative_attention_max_distance
        self.d_model = config.d_model
        self.key_value_proj_dim = config.d_kv
        self.n_heads = config.num_heads
        self.dropout = config.dropout_rate
        self.inner_dim = self.n_heads * self.key_value_proj_dim

        self.q = nn.Linear(self.d_model, self.inner_dim, bias=False)
        self.k = nn.Linear(self.d_model, self.inner_dim, bias=False)
        self.v = nn.Linear(self.d_model, self.inner_dim, bias=False)
        self.o = nn.Linear(self.inner_dim, self.d_model, bias=False)

        '''
        Here is where the change lies, i.e adding the relative_horizontal_bias as well as the relative_vertical_bias
        '''
        if self.has_relative_attention_bias:
            self.relative_attention_bias = nn.Embedding(
                self.relative_attention_num_buckets, self.n_heads)
            self.relative_horizontal_bias = nn.Embedding(
                self.relative_attention_num_buckets, self.n_heads)
            self.relative_vertical_bias = nn.Embedding(
                self.relative_attention_num_buckets, self.n_heads)

        self.gradient_checkpointing = False

    @staticmethod
    def _relative_position_bucket(relative_position, bidirectional=True, num_buckets=32, max_distance=128):
        """
        Adapted from Mesh Tensorflow:
        https://github.com/tensorflow/mesh/blob/0cb87fe07da627bf0b7e60475d59f95ed6b5be3d/mesh_tensorflow/transformer/transformer_layers.py#L593
        Translate relative position to a bucket number for relative attention. The relative position is defined as
        memory_position - query_position, i.e. the distance in tokens from the attending position to the attended-to
        position. If bidirectional=False, then positive relative positions are invalid. We use smaller buckets for
        small absolute relative_position and larger buckets for larger absolute relative_positions. All relative
        positions >=max_distance map to the same bucket. All relative positions <=-max_distance map to the same bucket.
        This should allow for more graceful generalization to longer sequences than the model has been trained on
        Args:
            relative_position: an int32 Tensor
            bidirectional: a boolean - whether the attention is bidirectional
            num_buckets: an integer
            max_distance: an integer
        Returns:
            a Tensor with the same shape as relative_position, containing int32 values in the range [0, num_buckets)
        """
        relative_buckets = 0
        if bidirectional:
            num_buckets //= 2
            relative_buckets += (relative_position
                                 > 0).to(torch.long) * num_buckets
            relative_position = torch.abs(relative_position)
        else:
            relative_position = - \
                torch.min(relative_position,
                          torch.zeros_like(relative_position))
        # now relative_position is in the range [0, inf)

        # half of the buckets are for exact increments in positions
        max_exact = num_buckets // 2
        is_small = relative_position < max_exact

        # The other half of the buckets are for logarithmically bigger bins in positions up to max_distance
        relative_position_if_large = max_exact + (
            torch.log(relative_position.float() / max_exact)
            / math.log(max_distance / max_exact)
            * (num_buckets - max_exact)
        ).to(torch.long)
        relative_position_if_large = torch.min(
            relative_position_if_large, torch.full_like(
                relative_position_if_large, num_buckets - 1)
        )

        relative_buckets += torch.where(is_small,
                                        relative_position, relative_position_if_large)
        return relative_buckets

    def compute_bias_1d(self, query_length, key_length, device=None):
        """Compute binned relative position bias"""
        if device is None:
            device = self.relative_attention_bias.weight.device
        context_position = torch.arange(
            query_length, dtype=torch.long, device=device)[:, None]
        memory_position = torch.arange(
            key_length, dtype=torch.long, device=device)[None, :]
        relative_position = memory_position - \
            context_position  # shape (query_length, key_length)
        relative_position_bucket = self._relative_position_bucket(
            relative_position,  # shape (query_length, key_length)
            bidirectional=(not self.is_decoder),
            num_buckets=self.relative_attention_num_buckets,
            max_distance=self.relative_attention_max_distance,
        )
        # shape (query_length, key_length, num_heads)
        values = self.relative_attention_bias(relative_position_bucket)
        # shape (1, num_heads, query_length, key_length)
        values = values.permute([2, 0, 1]).unsqueeze(0)
        return values

    def compute_vertical_horizontal_bias(self, total_boxes: int = 512, device=None):

        denominator_to_divide = total_boxes // self.relative_attention_num_buckets

        """Compute the vertical and horizontal bias"""
        if device is None:
            device = self.relative_attention_bias.weight.device
        indices = torch.arange(total_boxes, dtype=torch.long, device=device)
        h_distances = (indices % self.relative_attention_num_buckets)[
            :, None] - (indices % self.relative_attention_num_buckets)[None, :]
        v_distances = (
            indices // denominator_to_divide)[:, None] - (indices // denominator_to_divide)[None, :]

        h_distances_bucket = self._relative_position_bucket(
            h_distances,  # shape (query_length, key_length)
            bidirectional=(not self.is_decoder),
            num_buckets=self.relative_attention_num_buckets,
            max_distance=self.relative_attention_max_distance,
        )

        ## It has to be like this : https://github.com/microsoft/i-Code/blob/d933ae53eb9dec057e605fa4c89ea701629c5b9d/i-Code-Doc/core/models/embedding/relative/relative.py#L175
        ## so change is needed here
        v_distances_bucket = self._relative_position_bucket(
            v_distances,  # shape (query_length, key_length)
            bidirectional=(not self.is_decoder),
            num_buckets=self.relative_attention_num_buckets,
            max_distance=self.relative_attention_max_distance,
        )

        h_distances_values = self.relative_horizontal_bias(
            h_distances_bucket)  # shape (query_length, key_length, num_heads)
        h_distances_values = h_distances_values.permute([2, 0, 1]).unsqueeze(
            0)  # shape (1, num_heads, query_length, key_length)

        v_distances_values = self.relative_vertical_bias(
            v_distances_bucket)  # shape (query_length, key_length, num_heads)
        v_distances_values = v_distances_values.permute([2, 0, 1]).unsqueeze(
            0)  # shape (1, num_heads, query_length, key_length)

        return h_distances_values, v_distances_values

    def forward(self, hidden_states, mask=None, key_value_states=None, position_bias=None, past_key_value=None, layer_head_mask=None, query_length=None,
                use_cache=False, output_attentions=False):
        """
        Self-attention (if key_value_states is None) or attention over source sentence (provided by key_value_states).
        """
        # Input is (batch_size, seq_length, dim)
        # Mask is (batch_size, key_length) (non-causal) or (batch_size, key_length, key_length)
        # past_key_value[0] is (batch_size, n_heads, q_len - 1, dim_per_head)
        batch_size, seq_length = hidden_states.shape[:2]

        real_seq_length = seq_length

        if past_key_value is not None:
            assert(len(past_key_value)
                   == 2), f"past_key_value should have 2 past states: keys and values. Got { len(past_key_value)} past states"
            real_seq_length += past_key_value[0].shape[2] if query_length is None else key_value_states.shape[1]

        key_length = real_seq_length if key_value_states is None else key_value_states.shape[
            1]

        def shape(states):
            "projection"
            return states.view(batch_size, -1, self.n_heads, self.key_value_proj_dim).transpose(1, 2)

        def unshape(states):
            """reshape"""
            return states.transpose(1, 2).contiguous().view(batch_size, -1, self.inner_dim)

        def project(hidden_states, proj_layer, key_value_states, past_key_value):
            """project hidden states correctly to key/query states"""
            if key_value_states is None:
                # self-attn
                # (batch_size, n_heads, seq_length, dim_per_head)
                hidden_states = shape(proj_layer(hidden_states))
            elif past_key_value is None:
                # cross-attn
                # (batch_size, n_heads, seq_length, dim_per_head)
                hidden_states = shape(proj_layer(key_value_states))

            if past_key_value is not None:
                if key_value_states is None:
                    # self-attn
                    # (batch_size, n_heads, key_length, dim_per_head)
                    hidden_states = torch.cat(
                        [past_key_value, hidden_states], dim=2)
                elif past_key_value.shape[2] != key_value_states.shape[1]:
                    # checking that the `sequence_length` of the `past_key_value` is the same as
                    # the provided `key_value_states` to support prefix tuning
                    # cross-attn
                    # (batch_size, n_heads, seq_length, dim_per_head)
                    hidden_states = shape(proj_layer(key_value_states))
                else:
                    # cross-attn
                    hidden_states = past_key_value
            return hidden_states

        # get query states
        query_states = shape(self.q(hidden_states))

        # get key/value states
        key_states = project(hidden_states, self.k, key_value_states,
                             past_key_value[0] if past_key_value is not None else None)
        value_states = project(hidden_states, self.v, key_value_states,
                               past_key_value[0] if past_key_value is not None else None)

        # compute score
        # equivalent of torch.einsum("bnqd,bnkd->bnqk", query_states, key_states), compatible with onnx op>9
        scores = torch.matmul(query_states, key_states.transpose(3, 2))

        # Sequential Part
        if position_bias is None:
            if not self.has_relative_attention_bias:
                position_bias = torch.zeros(
                    (1, self.n_heads, real_seq_length, key_length), device=scores.device, dtype=scores.dtype
                )
                if self.gradient_checkpointing and self.training:
                    position_bias.requires_grad = True
            else:
                position_bias = self.compute_bias_1d(
                    real_seq_length, key_length, device=scores.device)
                h_distances_values, v_distances_values = self.compute_vertical_horizontal_bias(
                    total_boxes=real_seq_length, device=scores.device)
                position_bias = position_bias + h_distances_values + v_distances_values

            # if key and values are already calculated
            # we want only the last query position bias
            if past_key_value is not None:
                position_bias = position_bias[:, :, -hidden_states.size(1):, :]

            if mask is not None:
                # position_bias = position_bias + mask # (batch_size, n_heads, seq_length, key_length)
                # Arctic-TILT fix: Ensure mask dimensions match position_bias
                if mask.dim() == 2:
                    # Convert 2D mask to 4D: (batch_size, seq_len) -> (batch_size, 1, 1, seq_len)
                    mask = mask.unsqueeze(1).unsqueeze(1)
                elif mask.dim() == 3:
                    # Convert 3D mask to 4D: (batch_size, 1, seq_len) -> (batch_size, 1, 1, seq_len)
                    mask = mask.unsqueeze(1)

                # Expand mask to match position_bias shape if needed
                if mask.shape[-1] != position_bias.shape[-1] or mask.shape[-2] != position_bias.shape[-2]:
                    # Expand the mask to match the position_bias dimensions
                    mask = mask.expand(-1, -1, position_bias.shape[-2], position_bias.shape[-1])

                # Ensure mask has the same batch size as position_bias
                if mask.shape[0] != position_bias.shape[0] and position_bias.shape[0] == 1:
                    # position_bias is broadcasted, expand it to match batch size
                    position_bias = position_bias.expand(mask.shape[0], -1, -1, -1)
                elif mask.shape[0] == 1 and position_bias.shape[0] != 1:
                    # mask needs to be broadcasted
                    mask = mask.expand(position_bias.shape[0], -1, -1, -1)

                # Add mask to position bias
                position_bias = position_bias + mask

        position_bias_masked = position_bias  # No pruning right now

        scores += position_bias_masked
        attn_weights = nn.functional.softmax(scores.float(), dim=-1).type_as(
            scores
        )  # (batch_size, n_heads, seq_length, key_length)
        attn_weights = nn.functional.dropout(
            attn_weights, p=self.dropout, training=self.training
        )  # (batch_size, n_heads, seq_length, key_length)

        # Mask heads if we want to
        if layer_head_mask is not None:
            attn_weights = attn_weights * layer_head_mask

        # (batch_size, seq_length, dim)
        attn_output = unshape(torch.matmul(attn_weights, value_states))
        attn_output = self.o(attn_output)

        present_key_value_state = (key_states, value_states) if (
            self.is_decoder and use_cache) else None
        outputs = (attn_output,) + \
            (present_key_value_state,) + (position_bias,)

        if output_attentions:
            outputs = outputs + (attn_weights,)
        return outputs


class T5LayerSelfAttention(nn.Module):
    def __init__(self, config, has_relative_attention_bias=False):
        super().__init__()
        self.SelfAttention = T5Attention(
            config, has_relative_attention_bias=has_relative_attention_bias)
        self.layer_norm = T5LayerNorm(
            config.d_model, eps=config.layer_norm_epsilon)
        self.dropout = nn.Dropout(config.dropout_rate)

    def forward(self, hidden_states, attention_mask=None, position_bias=None, layer_head_mask=None, past_key_value=None, use_cache=False, output_attentions=False):
        normed_hidden_states = self.layer_norm(hidden_states)
        attention_output = self.SelfAttention(normed_hidden_states, mask=attention_mask, position_bias=position_bias,
                                              layer_head_mask=layer_head_mask, past_key_value=past_key_value, use_cache=use_cache, output_attentions=output_attentions,)
        hidden_states = hidden_states + self.dropout(attention_output[0])
        # add attentions if we output them
        outputs = (hidden_states,) + attention_output[1:]
        return outputs


class T5LayerCrossAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.EncDecAttention = T5Attention(
            config, has_relative_attention_bias=False)
        self.layer_norm = T5LayerNorm(
            config.d_model, eps=config.layer_norm_epsilon)
        self.dropout = nn.Dropout(config.dropout_rate)

    def forward(self, hidden_states, key_value_states, attention_mask=None, position_bias=None, layer_head_mask=None, past_key_value=None, use_cache=False, query_length=None, output_attentions=False, ):
        normed_hidden_states = self.layer_norm(hidden_states)
        attention_output = self.EncDecAttention(normed_hidden_states, mask=attention_mask,
                                                key_value_states=key_value_states, position_bias=position_bias,
                                                layer_head_mask=layer_head_mask,
                                                past_key_value=past_key_value,
                                                use_cache=use_cache,
                                                query_length=query_length,
                                                output_attentions=output_attentions,)
        layer_output = hidden_states + self.dropout(attention_output[0])
        # add attention if we output them
        outputs = (layer_output, ) + attention_output[1:]
        return outputs


class T5Block(nn.Module):
    def __init__(self, config, has_relative_attention_bias=False):
        super().__init__()
        self.is_decoder = config.is_decoder
        self.layer = nn.ModuleList()
        self.layer.append(T5LayerSelfAttention(
            config, has_relative_attention_bias=has_relative_attention_bias))
        if self.is_decoder:
            self.layer.append(T5LayerCrossAttention(config))

        self.layer.append(T5LayerFF(config))

        # Arctic-TILT: Always include post-fusion module
        fusion_layer_norm = TiltLayerNorm(config.d_model)
        fusion_dropout = getattr(config, 'fusion_dropout', 0.1)
        self.post_fusion = TiltPostFusionModule(
            d_model=config.d_model,
            dropout=fusion_dropout,
            layer_norm=fusion_layer_norm
        )

    def forward(self, hidden_states, attention_mask=None, position_bias=None, encoder_hidden_states=None,
                encoder_attention_mask=None, encoder_decoder_position_bias=None, layer_head_mask=None, cross_attn_layer_head_mask=None,
                past_key_value=None, use_cache=False, output_attentions=False, return_dict=True, vision_embeddings=None):

        if past_key_value is not None:
            expected_num_past_key_values = 2 if encoder_hidden_states is None else 4

            if len(past_key_value) != expected_num_past_key_values:
                raise ValueError(
                    f"There should be {expected_num_past_key_values} past states. "
                    f"{'2 (past / key) for cross attention. ' if expected_num_past_key_values == 4 else ''}"
                    f"Got {len(past_key_value)} past key / value states"
                )

            self_attn_past_key_value = past_key_value[:2]
            cross_attn_past_key_value = past_key_value[2:]
        else:
            self_attn_past_key_value, cross_attn_past_key_value = None, None

        self_attention_outputs = self.layer[0](
            hidden_states,
            attention_mask=attention_mask,
            position_bias=position_bias,
            layer_head_mask=layer_head_mask,
            past_key_value=self_attn_past_key_value,
            use_cache=use_cache,
            output_attentions=output_attentions,
        )
        hidden_states, present_key_value_state = self_attention_outputs[:2]
        # Keep self-attention outputs and relative position weights
        attention_outputs = self_attention_outputs[2:]

        # clamp inf values to enable fp16 training
        if hidden_states.dtype == torch.float16 and torch.isinf(hidden_states).any():
            clamp_value = torch.finfo(hidden_states.dtype).max - 1000
            hidden_states = torch.clamp(
                hidden_states, min=-clamp_value, max=clamp_value)

        do_cross_attention = self.is_decoder and encoder_hidden_states is not None
        if do_cross_attention:
            # the actual query length is unknown for cross attention
            # if using past key value states. Need to inject it here
            if present_key_value_state is not None:
                query_length = present_key_value_state[0].shape[2]
            else:
                query_length = None

            cross_attention_outputs = self.layer[1](
                hidden_states,
                key_value_states=encoder_hidden_states,
                attention_mask=encoder_attention_mask,
                position_bias=encoder_decoder_position_bias,
                layer_head_mask=cross_attn_layer_head_mask,
                past_key_value=cross_attn_past_key_value,
                query_length=query_length,
                use_cache=use_cache,
                output_attentions=output_attentions,
            )
            hidden_states = cross_attention_outputs[0]

            # clamp inf values to enable fp16 training
            if hidden_states.dtype == torch.float16 and torch.isinf(hidden_states).any():
                clamp_value = torch.finfo(hidden_states.dtype).max - 1000
                hidden_states = torch.clamp(
                    hidden_states, min=-clamp_value, max=clamp_value)

            # Combine self attn and cross attn key value states
            if present_key_value_state is not None:
                present_key_value_state = present_key_value_state + \
                    cross_attention_outputs[1]

            # Keep cross-attention outputs and relative position weights
            attention_outputs = attention_outputs + cross_attention_outputs[2:]

        # Apply Feed Forward layer
        hidden_states = self.layer[-1](hidden_states)

        # Arctic-TILT: Apply post-fusion when vision embeddings are provided
        if vision_embeddings is not None:
            hidden_states = self.post_fusion(hidden_states, vision_embeddings)

        # clamp inf values to enable fp16 training
        if hidden_states.dtype == torch.float16 and torch.isinf(hidden_states).any():
            clamp_value = torch.finfo(hidden_states.dtype).max - 1000
            hidden_states = torch.clamp(
                hidden_states, min=-clamp_value, max=clamp_value)

        outputs = (hidden_states,)

        if use_cache:
            outputs = outputs + (present_key_value_state,) + attention_outputs
        else:
            outputs = outputs + attention_outputs

        # hidden-states, present_key_value_states, (self-attention position bias), (self-attention weights), (cross-attention position bias), (cross-attention weights)
        return outputs


class T5Stack(t5.modeling_t5.T5Stack):
    def __init__(self, config, embed_tokens=None):
        '''Arctic-TILT implementation with custom T5Block and chunked processing'''
        super().__init__(config=config, embed_tokens=embed_tokens)
        self.block = nn.ModuleList(
            [T5Block(config, has_relative_attention_bias=bool(i == 0))
             for i in range(config.num_layers)]
        )

        # Initialize chunked processor for Arctic-TILT
        if getattr(config, 'use_chunked_processing', False):
            self.chunked_processor = ChunkedProcessor(
                core_chunk_length=getattr(config, 'core_chunk_length', 1024),
                chunk_overlap=getattr(config, 'chunk_overlap', 0)
            )
        else:
            self.chunked_processor = None

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        inputs_embeds=None,
        head_mask=None,
        cross_attn_head_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        vision_embeddings=None,
    ):
        # Arctic-TILT: Check if chunked processing should be used
        use_chunked = (
            self.chunked_processor is not None and
            not self.is_decoder and  # Only for encoder
            input_ids is not None and
            input_ids.shape[1] > self.chunked_processor.core_chunk_length
        )

        if use_chunked:
            # Use chunked processing for long sequences in encoder
            return self._forward_chunked(
                input_ids=input_ids,
                attention_mask=attention_mask,
                inputs_embeds=inputs_embeds,
                head_mask=head_mask,
                use_cache=use_cache,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
                vision_embeddings=vision_embeddings,
            )
        else:
            # Use standard processing
            return self._forward_standard(
                input_ids=input_ids,
                attention_mask=attention_mask,
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_attention_mask,
                inputs_embeds=inputs_embeds,
                head_mask=head_mask,
                cross_attn_head_mask=cross_attn_head_mask,
                past_key_values=past_key_values,
                use_cache=use_cache,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
                vision_embeddings=vision_embeddings,
            )

    def _forward_chunked(
        self,
        input_ids=None,
        attention_mask=None,
        inputs_embeds=None,
        head_mask=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        vision_embeddings=None,
    ):
        """
        Forward pass with chunked processing for long sequences.
        Implements SLED approach with block diagonal attention.
        """
        # Set defaults
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        batch_size, seq_length = input_ids.shape
        device = input_ids.device

        # Create chunks using chunked processor
        chunked_inputs, chunked_masks, chunk_info = self.chunked_processor.create_chunks(
            input_ids, attention_mask, prefix_length=1
        )

        # Process each chunk separately
        chunk_outputs = []
        for chunk_input, chunk_mask in zip(chunked_inputs, chunked_masks):
            # Process single chunk through standard forward
            chunk_result = self._forward_standard(
                input_ids=chunk_input,
                attention_mask=chunk_mask,
                inputs_embeds=None,
                head_mask=head_mask,
                use_cache=False,  # Disable cache for chunks
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=True,
                vision_embeddings=vision_embeddings,
            )

            chunk_outputs.append(chunk_result.last_hidden_state)

        # Recombine chunk outputs
        combined_hidden_states = self.chunked_processor.recombine_chunk_outputs(
            chunk_outputs, prefix_length=1
        )

        # Return in appropriate format
        if return_dict:
            from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions
            return BaseModelOutputWithPastAndCrossAttentions(
                last_hidden_state=combined_hidden_states,
                past_key_values=None,
                hidden_states=None,
                attentions=None,
                cross_attentions=None,
            )
        else:
            return (combined_hidden_states,)

    def _forward_standard(
        self,
        input_ids=None,
        attention_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        inputs_embeds=None,
        head_mask=None,
        cross_attn_head_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        vision_embeddings=None,
    ):
        """
        Standard forward pass for T5Stack with vision embeddings fusion.
        """
        # Arctic-TILT: Custom forward pass with vision_embeddings passed to each block
        if vision_embeddings is not None:
            # Prepare inputs similar to original T5Stack forward
            use_cache = use_cache if use_cache is not None else self.config.use_cache
            output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
            output_hidden_states = (
                output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
            )
            return_dict = return_dict if return_dict is not None else self.config.use_return_dict

            if input_ids is not None and inputs_embeds is not None:
                raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
            elif input_ids is not None:
                input_shape = input_ids.size()
                input_ids = input_ids.view(-1, input_shape[-1])
            elif inputs_embeds is not None:
                input_shape = inputs_embeds.size()[:-1]
            else:
                raise ValueError("You have to specify either input_ids or inputs_embeds")

            if inputs_embeds is None:
                assert self.embed_tokens is not None, "You have to initialize the model with valid token embeddings"
                inputs_embeds = self.embed_tokens(input_ids)

            batch_size, seq_length = input_shape
            device = inputs_embeds.device

            # Prepare attention mask
            if attention_mask is None:
                attention_mask = torch.ones(batch_size, seq_length, device=device)

            # Arctic-TILT: Convert attention mask to extended format for position bias compatibility
            if attention_mask is not None:
                # Convert to float and apply mask logic: 1 for tokens to attend, 0 for tokens to ignore
                extended_attention_mask = attention_mask[:, None, None, :]
                extended_attention_mask = extended_attention_mask.to(dtype=inputs_embeds.dtype)
                extended_attention_mask = (1.0 - extended_attention_mask) * torch.finfo(inputs_embeds.dtype).min
            else:
                extended_attention_mask = None

            # Initialize past_key_values with empty states if not provided
            if past_key_values is None:
                past_key_values = [None] * len(self.block)

            # Prepare head mask if provided
            if head_mask is not None:
                if head_mask.dim() == 1:
                    head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
                    head_mask = head_mask.expand(self.config.num_layers, -1, -1, -1, -1)
                elif head_mask.dim() == 2:
                    head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)

            present_key_value_states = () if use_cache else None
            all_hidden_states = () if output_hidden_states else None
            all_attentions = () if output_attentions else None
            all_cross_attentions = () if (output_attentions and self.is_decoder) else None
            position_bias = None
            encoder_decoder_position_bias = None

            hidden_states = inputs_embeds

            # Pass through each block with vision embeddings
            for i, (layer_module, past_key_value) in enumerate(zip(self.block, past_key_values)):
                layer_head_mask = head_mask[i] if head_mask is not None else None
                cross_attn_layer_head_mask = None
                if self.is_decoder and encoder_hidden_states is not None:
                    cross_attn_layer_head_mask = head_mask[i] if head_mask is not None else None

                if output_hidden_states:
                    all_hidden_states = all_hidden_states + (hidden_states,)

                # Arctic-TILT: Pass vision_embeddings to each block for fusion
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask=extended_attention_mask,  # Use properly formatted attention mask
                    position_bias=position_bias,
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    encoder_decoder_position_bias=encoder_decoder_position_bias,
                    layer_head_mask=layer_head_mask,
                    cross_attn_layer_head_mask=cross_attn_layer_head_mask,
                    past_key_value=past_key_value,
                    use_cache=use_cache,
                    output_attentions=output_attentions,
                    vision_embeddings=vision_embeddings,  # Pass vision embeddings to each block
                )

                # layer_outputs is a tuple: (hidden-states, key-value-states, attention_weights, position_bias, ...)
                if use_cache is False:
                    layer_outputs = layer_outputs[:1] + (None,) + layer_outputs[1:]

                hidden_states, present_key_value_state = layer_outputs[:2]

                # Update position bias for next layer
                if i == 0:
                    position_bias = layer_outputs[2] if len(layer_outputs) > 2 else None
                    if self.is_decoder and encoder_hidden_states is not None:
                        encoder_decoder_position_bias = layer_outputs[3] if len(layer_outputs) > 3 else None

                if use_cache:
                    present_key_value_states = present_key_value_states + (present_key_value_state,)

                if output_attentions:
                    all_attentions = all_attentions + (layer_outputs[3] if len(layer_outputs) > 3 else layer_outputs[2],)
                    if self.is_decoder and encoder_hidden_states is not None:
                        all_cross_attentions = all_cross_attentions + (layer_outputs[4] if len(layer_outputs) > 4 else None,)

            hidden_states = self.final_layer_norm(hidden_states)
            hidden_states = self.dropout(hidden_states)

            # Add last hidden state
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            if not return_dict:
                return tuple(
                    v
                    for v in [
                        hidden_states,
                        present_key_value_states,
                        all_hidden_states,
                        all_attentions,
                        all_cross_attentions,
                    ]
                    if v is not None
                )

            # Return appropriate output class
            from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions
            return BaseModelOutputWithPastAndCrossAttentions(
                last_hidden_state=hidden_states,
                past_key_values=present_key_value_states,
                hidden_states=all_hidden_states,
                attentions=all_attentions,
                cross_attentions=all_cross_attentions,
            )
        else:
            # Fallback to standard forward if no vision embeddings
            return super().forward(
                input_ids=input_ids,
                attention_mask=attention_mask,
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_attention_mask,
                inputs_embeds=inputs_embeds,
                head_mask=head_mask,
                cross_attn_head_mask=cross_attn_head_mask,
                past_key_values=past_key_values,
                use_cache=use_cache,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict
            )


class T5Model(t5.modeling_t5.T5Model):
    def __init__(self, config):
        super().__init__(config=config)

        self.config = config
        encoder_config = copy.deepcopy(config)
        decoder_config = copy.deepcopy(config)
        decoder_config.update(dict(is_decoder=True))

        self.encoder = T5Stack(encoder_config, self.shared)
        self.decoder = T5Stack(decoder_config, self.shared)

        self.post_init()

    def forward(self, **kwargs):
        # Extract vision_embeddings if present
        vision_embeddings = kwargs.pop('vision_embeddings', None)

        # If we have vision embeddings, pass them to encoder
        if vision_embeddings is not None and 'encoder_outputs' not in kwargs:
            # Get encoder outputs with vision embeddings
            encoder_kwargs = {k: v for k, v in kwargs.items()
                            if k in ['input_ids', 'attention_mask', 'inputs_embeds', 'head_mask',
                                   'output_attentions', 'output_hidden_states', 'return_dict']}
            encoder_kwargs['vision_embeddings'] = vision_embeddings
            encoder_outputs = self.encoder(**encoder_kwargs)
            kwargs['encoder_outputs'] = encoder_outputs

        return super().forward(**kwargs)

    def load_weights(self):
        dummy_model = AutoModel.from_pretrained(self.config._name_or_path)
        self.load_state_dict(dummy_model.state_dict(), strict=False)
        print("Weights loaded successfully!")


class T5ForConditionalGeneration(t5.modeling_t5.T5ForConditionalGeneration):
    def __init__(self, config):
        '''
        It is similar to the T5ForConditionalGeneration described in the `hugging_face` repository, however I had to tweak it a bit,
        since there is an addition of the `relative_horizontal_bias` as well as `relative_vertical_bias` in the `T5Attention` class, and also
        the entire approach is generative in nature, so maybe it can be used in some other dataset, such as Question Answering
        '''

        super().__init__(config=config)

        self.config = config
        encoder_config = copy.deepcopy(config)
        decoder_config = copy.deepcopy(config)
        # In the pretrained version, the decoder config, the `is_decoder` option is True
        decoder_config.update(dict(is_decoder=True))

        self.encoder = T5Stack(encoder_config, self.shared)
        self.decoder = T5Stack(decoder_config, self.shared)

        if config.load_weights:
            self.load_weights()
        else:
            self.post_init()
            print("Initialization done without loading the weights")

    def forward(self, **kwargs):
        '''Same as mentioned in the hugging face's implementation'''
        return super().forward(**kwargs)

    def load_weights(self):
        '''
        Loads the weights from the pretrained model
        '''
        dummy_model = AutoModel.from_pretrained(self.config._name_or_path)
        self.load_state_dict(dummy_model.state_dict(), strict=False)
        print("Weights loaded successfully!")


class T5EncoderModel(t5.modeling_t5.T5ForConditionalGeneration):
    def __init__(self, config):
        '''
        It is similar to the T5EncoderModel described in the `hugging_face` repository, however I had to tweak it a bit,
        since there is an addition of the `relative_horizontal_bias` as well as `relative_vertical_bias` in the `T5Attention` class
        '''
        super().__init__(config=config)
        self.encoder = T5Stack(config, self.shared)
        self.post_init()

    def forward(self, **kwargs):
        '''Similar to the `T5EncoderModel` mentioned in the hugging face's t5 implementation'''
        return super().forward(**kwargs)


class T5ForConditionalGenerationAbstractive(t5.modeling_t5.T5ForConditionalGeneration):
    def __init__(self, config):
        '''
        T5ForConditionalGenerationAbstractive is a T5ForConditionalGeneration model with a linear layer on top of the decoder output,
        where the decoder output is the output of the last layer of the decoder, followed by a linear layer projection.

        It is similar to T5ForConditionalGeneration, however, it is based on a concept of generative answer, and this was what I did earlier,
        however, the authors have used an abstractive approach, and so I had to tweak somethinig, and essentially, it is the `self.lm_head`
        '''

        super().__init__(config=config)

        self.config = config
        encoder_config = copy.deepcopy(config)
        decoder_config = copy.deepcopy(config)
        # In the pretrained version, the decoder config, the `is_decoder` option is True
        decoder_config.update(dict(is_decoder=True))

        self.encoder = T5Stack(encoder_config, self.shared)
        self.decoder = T5Stack(decoder_config, self.shared)
        self.lm_head = nn.Linear(in_features=config.d_model,
                                 out_features=config.num_classes, bias=False)

        if config.load_weights:
            self.load_weights()
        else:
            self.post_init()
            print("Initialization done without loading the weights")

    def forward(self, **kwargs):
        '''
        Forward pass of T5ForConditionalGenerationAbstractive. It is similar to T5ForConditionalGeneration, however, it is based on a concept of generative answer,
        and this was what I did earlier,
        '''
        # Extract vision_embeddings if present
        vision_embeddings = kwargs.pop('vision_embeddings', None)

        # If we have vision embeddings, pass them to encoder
        if vision_embeddings is not None and 'encoder_outputs' not in kwargs:
            # Get encoder outputs with vision embeddings
            encoder_kwargs = {k: v for k, v in kwargs.items()
                            if k in ['input_ids', 'attention_mask', 'inputs_embeds', 'head_mask',
                                   'output_attentions', 'output_hidden_states', 'return_dict']}
            encoder_kwargs['vision_embeddings'] = vision_embeddings
            encoder_outputs = self.encoder(**encoder_kwargs)
            kwargs['encoder_outputs'] = encoder_outputs

        return super().forward(**kwargs)

    def load_weights(self):
        '''
        Load the weights of the T5ForConditionalGenerationAbstractive model
        It is adaptable to both the `t5-base` and `t5-large` configuration settings
        '''
        dummy_model = AutoModel.from_pretrained(self.config._name_or_path)
        self.load_state_dict(dummy_model.state_dict(), strict=False)
        print("Weights loaded successfully!")

## 1.6 Writing the collate_fn for custom handling of the dataloader

In [ ]:
class CollateFn(object):
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer

  def __call__(self, list_of_ds):
    simple_keys = ["input_ids", "attention_mask", "bboxes", "pixel_values", "labels"]
    actual_batch = {}
    for key in simple_keys:
      actual_batch[key] = torch.stack([x[key] for x in list_of_ds])


    return actual_batch

## 2.1 Preparing the visual model

In [ ]:
class VisualEmbedding(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.unet_encoder = Unet_encoder(in_channels = config.in_channels, channels = config.channels, num_pool_layers = config.num_pool_layers)
    self.roi_pool = RoIPool(output_size = config.output_size, spatial_scale = config.spatial_scale)
    self.proj = nn.Linear(in_features = 128 * 3 * 3, out_features = config.d_model)
    self.config = config

  def forward(self, pixel_values, bboxes):
    image_embedding = self.unet_encoder(pixel_values)
    feature_maps_bboxes = self.roi_pool(image_embedding, bboxes).flatten(2)
    projection = self.proj(feature_maps_bboxes)
    return projection

## 2.2 Preparing the semantic model

In [ ]:
class TiLTTransformer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config
    self.visual_embedding_extractor = VisualEmbedding(config)
    self.t5_model = T5ForConditionalGenerationAbstractive(config)


  def common_step(self, batch):
    ## Visual embedding
    visual_embedding = self.visual_embedding_extractor(pixel_values=batch['pixel_values'], bboxes=batch['bboxes'])

    ## Semantic embedding from t5_model's embedding layer
    semantic_embedding = self.t5_model.shared(batch['input_ids'])

    # Arctic-TILT: Return both embeddings separately for fusion in each transformer block
    return semantic_embedding, visual_embedding

  def forward(self, batch):
    embeddings, vision_embeddings = self.common_step(batch)

    # Arctic-TILT: Pass vision embeddings to be used in each transformer block
    final_output = self.t5_model(
        attention_mask=batch['attention_mask'],
        inputs_embeds=embeddings,
        labels=batch['labels'],
        vision_embeddings=vision_embeddings
    )

    return final_output

## 3.1 Preparing the metrics to evaluate the predictions

In [ ]:
import evaluate

def get_labels(predictions, references):

    # Transform predictions and references tensors to numpy arrays
    if predictions.device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()

    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

## Part: 4 Writing the pytorch_lightning code for training on FUNSD

In [ ]:
id2label, label2id = get_id2label_and_label2id()
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambda x : 2 * x - 1)])

## Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True, model_max_length = model_max_length)

train_ds = ExtFUNSDDs(hf_ds['train'],tokenizer = tokenizer, transform = transform)
val_ds = ExtFUNSDDs(hf_ds['test'],tokenizer = tokenizer, transform = transform)

collate_fn = CollateFn(tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
class DataModule(pl.LightningDataModule):

  def __init__(self, train_dataset, eval_dataset, batch_size: int = 2):
    super(DataModule, self).__init__()
    self.batch_size = batch_size
    self.train_dataset = train_dataset
    self.eval_dataset = eval_dataset

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size,
                      shuffle=True, collate_fn = collate_fn)

  def val_dataloader(self):
    return DataLoader(self.eval_dataset, batch_size=self.batch_size,
                      shuffle=False, collate_fn = collate_fn)

In [ ]:
class TiltModel(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.tilt_model = TiLTTransformer(config)
        self.eval_metric = evaluate.load("seqeval")

    def forward(self, batch):
        return self.tilt_model(batch)

    def training_step(self, batch, batch_idx):
        output = self(batch)
        loss = output.loss

        predictions = output.logits.argmax(dim=-1)
        true_predictions, true_labels = get_labels(predictions = predictions, references =  batch['labels'])

        results = self.eval_metric.compute(predictions=true_predictions, references=true_labels)
        self.log("train_loss", output.loss.item(),
                 prog_bar=True, on_epoch=True, logger=True)
        self.log("train_overall_fl",
                 results["overall_f1"], prog_bar=True, on_epoch=True, logger=True)
        self.log("train_overall_recall",
                 results["overall_recall"], prog_bar=True, on_epoch=True, logger=True)
        self.log("train_overall_precision",
                 results["overall_precision"], prog_bar=True, on_epoch=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        output = self(batch)
        loss = output.loss

        predictions = output.logits.argmax(dim=-1)
        true_predictions, true_labels = get_labels(predictions = predictions, references =  batch['labels'])
        results = self.eval_metric.compute(predictions=true_predictions, references=true_labels)

        self.log("val_loss", output.loss.item(),
                 prog_bar=True, on_epoch=True, logger=True)
        self.log("val_overall_fl", results["overall_f1"],
                 prog_bar=True, on_epoch=True, logger=True)
        self.log("val_overall_recall",
                 results["overall_recall"], prog_bar=True, on_epoch=True, logger=True)
        self.log("val_overall_precision",
                 results["overall_precision"], prog_bar=True, on_epoch=True, logger=True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.lr)
        return optimizer

In [ ]:
def perform_evaluation(path: str=None, pl_model=None, pl_dl=None):
    print("Evaluating the model")
    if path is not None:
      pl_model = pl_model.load_from_checkpoint(path, config = t5_config)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    eval_metric = evaluate.load("seqeval")
    pl_model = pl_model.to(device)
    pl_model.eval();

    for idx, batch in enumerate(tqdm(pl_dl.val_dataloader())):
      # move batch to device
      batch = {k: v.to(device) for k, v in batch.items()}

      with torch.no_grad():
        outputs = pl_model(batch)

        predictions = outputs.logits.argmax(-1)
        true_predictions, true_labels = get_labels(predictions, batch["labels"])
        eval_metric.add_batch(references=true_labels,predictions=true_predictions)

    results = eval_metric.compute()

    metrics = {}
    for key in ['overall_precision', 'overall_recall', 'overall_f1', 'overall_accuracy']:
      print_statement = '{0: <30}'.format(str(key) + " has value:")
      print(print_statement, results[key])
      metrics[key] = results[key]

    return metrics

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath="./tilt/models",monitor="val_overall_fl", mode="max", filename='tilt_best_ckpt', save_top_k = 1)
logger = CSVLogger("./tilt/logs", name="funsd_dataset")

In [ ]:
wandb.init(config=t5_config, project="TiLT on FUNSD")
wandb_logger = WandbLogger(project="TiLT on FUNSD", log_model = False, entity="iakarshu")

In [ ]:
trainer = pl.Trainer(default_root_dir="./tilt/logs", devices="auto",
                     accelerator="auto",max_epochs= t5_config.max_epochs,
                     logger=wandb_logger,callbacks=[checkpoint_callback],
                     log_every_n_steps = 5)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
pl_model = TiltModel(t5_config)
pl_dl = DataModule(train_ds, val_ds, batch_size = 2)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Weights loaded successfully!


In [ ]:
trainer.fit(pl_model, pl_dl)

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type            | Params | Mode 
-------------------------------------------------------
0 | tilt_model | TiLTTransformer | 266 M  | train
-------------------------------------------------------
266 M     Trainable params
0         Non-trainable params
266 M     Total params
1,067.082 Total estimated model params size (MB)
751       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training: |          | 0/? [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [ ]:
ckpt_folder = "./tilt/models"
if os.path.exists(ckpt_folder):
  ckpt_path = os.path.join(ckpt_folder, os.listdir(ckpt_folder)[0])
else:
  ckpt_path = None

In [ ]:
metrics = perform_evaluation(path = ckpt_path, pl_model = TiltModel, pl_dl = pl_dl)


Evaluating the model
Weights loaded successfully!


  0%|          | 0/25 [00:00<?, ?it/s]

overall_precision has value:   0.5359355638166047
overall_recall has value:      0.38999098286744815
overall_f1 has value:          0.4514613778705637
overall_accuracy has value:    0.814305870805472


In [ ]:
print(metrics)


{'overall_precision': np.float64(0.5359355638166047), 'overall_recall': np.float64(0.38999098286744815), 'overall_f1': np.float64(0.4514613778705637), 'overall_accuracy': 0.814305870805472}
